# Researcher view for exploring data

More code-orientied exploration of data, view for researchers.

First, fetch data for user.

In [ ]:
from enum import Enum

import pandas as pd
from jupyter_health import JupyterHealthCHClient

pd.options.mode.chained_assignment = None

ch_client = JupyterHealthCHClient()

patient_id = "test-user"

df = ch_client.fetch_data_frame(patient_id)
df.head()

Reduce data to relevant subset for blood pressure

In [ ]:
# reduce data
bp = df.loc[df.resource_type == 'BLOOD_PRESSURE', ['systolic_blood_pressure_value', 'diastolic_blood_pressure_value', 'effective_time_frame_date_time']]
bp = bp.astype({"systolic_blood_pressure_value": int, "diastolic_blood_pressure_value": int})
bp

Plot over time

In [ ]:
bp.plot(x="effective_time_frame_date_time", y=["diastolic_blood_pressure_value", "systolic_blood_pressure_value"])

Compute goals, categories

In [ ]:
class Goal(Enum):
    """Enum for met/unmet

    These strings will be used for the legend.
    """

    met = "goal"
    unmet = "over"


class Category(Enum):
    normal = "normal"
    elevated = "elevated"
    hypertension = "hypertension"


def classify_bp(row):
    """Classify blood pressure"""
    # https://www.heart.org/en/health-topics/high-blood-pressure/understanding-blood-pressure-readings
    # note from : We can decide to have just normal, elevated and hypertension to begin with
    if (
        row.diastolic_blood_pressure_value < 80
        and row.systolic_blood_pressure_value < 120
    ):
        return Category.normal.value
    elif (
        row.diastolic_blood_pressure_value < 80
        and 120 <= row.systolic_blood_pressure_value < 130
    ):
        return Category.elevated.value
    else:
        return Category.hypertension.value


def bp_goal(patient_df, goal="140/90"):
    """True/False for blood pressure met goal"""
    sys_goal, dia_goal = (int(s) for s in goal.split("/"))
    if (patient_df.systolic_blood_pressure_value <= sys_goal) & (
        patient_df.diastolic_blood_pressure_value <= dia_goal
    ):
        return Goal.met.value
    else:
        return Goal.unmet.value


bp["category"] = bp.apply(classify_bp, axis=1)
bp["goal"] = bp.apply(bp_goal, axis=1)
bp

Compute fractions by category, goal

In [ ]:
bp.category.value_counts(normalize=True)

In [ ]:
bp.goal.value_counts(normalize=True)

Test out styling based on therapeutic goal 

In [ ]:
def bp_goal_style(row):
    """highlight rows exceeding bp goal"""
    goal = Goal(row.goal)
    if goal == Goal.unmet:
        color = "#fdd"
    else:
        color = None
    
    return [f"background-color:{color}" if color else None] * len(row)


bp[-50:].style.hide().hide(["category", "goal"], axis="columns").apply(bp_goal_style, axis=1)